## Word Vectorization on German article dataset

In [18]:
import nltk
import random 
import pandas as pd
import numpy as np
import re

from nltk.corpus import stopwords

from gensim.models import Word2Vec
from gensim.models import Phrases
from gensim.models.phrases import Phraser

import pickle

## Exploring the Data

In [19]:
data = pd.read_csv('articles/articles.csv', sep='\n', names=["Articles"], header=None)

In [20]:
data.head()

,Articles
0,Etat;Die ARD-Tochter Degeto hat sich verpflich...
1,Etat;App sei nicht so angenommen worden wie ge...
2,Etat;'Zum Welttag der Suizidprävention ist es ...
3,Etat;Mitarbeiter überreichten Eigentümervertre...
4,Etat;Service: Jobwechsel in der Kommunikations...


In [21]:
data = pd.DataFrame(data.Articles.str.split(';',1).tolist(), 
                         columns = ['Category','Text'])

In [22]:
data.head()

,Category,Text
0,Etat,"Die ARD-Tochter Degeto hat sich verpflichtet, ..."
1,Etat,App sei nicht so angenommen worden wie geplant...
2,Etat,"'Zum Welttag der Suizidprävention ist es Zeit,..."
3,Etat,Mitarbeiter überreichten Eigentümervertretern ...
4,Etat,Service: Jobwechsel in der Kommunikationsbranc...


In [23]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to C:\Users\Sven-
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [24]:
# get stop words from nltk
stopWords = stopwords.words('german')

# pre processing data
def cleanData(sentence):
    processedList = ""
    
    # convert to lowercase, ignore all special characters - keep only alpha-numericals and spaces (not removing full-stop here)
    sentence = re.sub(r'[^a-zA-ZäöüÄÖÜß0-9\s.]',r'',str(sentence))
    sentence = re.sub(r'\n',r' ',sentence)
    
    # remove stop words - not working for German
    #sentence = " ".join([word for word in sentence.split() if word not in stopWords])
    
    return sentence

In [25]:
cleanData(data['Text'][2])

'Zum Welttag der Suizidprävention ist es Zeit das alte Dogma Über Suizide schreibt man nicht zu verwerfen. Denn erst Ausnahmen von dieser Regel führen zu Nachahmungen  und wer über Suizidalität dann schweigt wenn sie nicht sensationell ist stigmatisiert die Betroffenen weiter. Halbwissen ist oft gefährlicher als Nichtwissen. WertherEffekt Ja ja schon mal gehört Wird über einen Suizid berichtet töten sich in der Folge mehr Menschen selbst. So wie damals als Goethes Briefroman Die Leiden des jungen Werthers erschien. Also Über Suizide wird nicht berichtet. Stimmt. Fast. Der 10. September ist Welttag der Suizidprävention. Wenn es darum geht Suizide zu verhinden müssen auch wir Journalisten unsere Verantwortung wahrnehmen  das bedeutet in diesem Fall sich vom alten Stehsatz Über Suizide schreibt man nicht zu verabschieden. Das Dogma des Schreibverbots hält sich unter Journalisten. Dabei gibt es drei gute Gründe dafür es aus den Redaktionen zu verbannen. Eines nach dem anderen. Tatsächlich 

In [26]:
# clean data 
data['Text'] = data['Text'].map(lambda x: cleanData(x))

In [27]:
data.head()

,Category,Text
0,Etat,Die ARDTochter Degeto hat sich verpflichtet ab...
1,Etat,App sei nicht so angenommen worden wie geplant...
2,Etat,Zum Welttag der Suizidprävention ist es Zeit d...
3,Etat,Mitarbeiter überreichten Eigentümervertretern ...
4,Etat,Service Jobwechsel in der Kommunikationsbranch...


# Word2Vec

In [28]:
tmp_corpus = data['Text'].map(lambda x: x.split('.'))

In [29]:
corpus = []
for i in range(len(tmp_corpus)):
    for line in tmp_corpus[i]:
        words = [x for x in line.split()]
        corpus.append(words)

In [30]:
num_of_sentences = len(corpus)
num_of_words = 0
for line in corpus:
    num_of_words += len(line)

print('Num of sentences - %s'%(num_of_sentences))
print('Num of words - %s'%(num_of_words))

Num of sentences - 258500
Num of words - 3721509


In [31]:
def shuffle_corpus(sentences):
    shuffled = list(sentences)
    random.shuffle(shuffled)
    return shuffled

In [32]:
# sg - skip gram |  window = size of the window | size = vector dimension
size = 110
window_size = 37 # sentences are long because they are articles
epochs = 200
min_count = 100
workers = 1000

# train word2vec model using gensim
model = Word2Vec(corpus, sg=1,window=window_size,size=size,
                 min_count=min_count,workers=workers,iter=epochs,sample=0.01)

In [33]:
model.build_vocab(sentences=shuffle_corpus(corpus),update=True)

for i in range(5):
    model.train(sentences=shuffle_corpus(corpus),epochs=30,total_examples=model.corpus_count)

In [34]:
model.wv.save_word2vec_format("model/embeddings.model", binary=True)

In [35]:
#model.wv.save_word2vec_format("submissions/embeddings30-625.model", binary=True)

In [36]:
# save model
#model.save('w2v_model')
#pickle.dump(model, open('embeddings.model', 'wb'))

In [37]:
# load word2vec model
#model = Word2Vec.load('w2v_model')

## Using the model

Word2vec training is an unsupervised task, there’s no good way to objectively evaluate the result. Evaluation depends on your end application.

In [ ]:
model.most_similar(positive=['Mann', 'König'], negative=['Frau'], topn=1)

In [ ]:
model.most_similar(positive=['glücklich', 'traurig'], negative=['nerven'], topn=2)